In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib import cm
from scipy.stats import norm
import pylab
import pandas as pd
import scipy.interpolate as interp
from mpl_toolkits.mplot3d import Axes3D
tf.compat.v1.disable_eager_execution()

/Users/shankar/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/shankar/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/shankar/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/shanka

In [2]:
dimension_state = 1

riskfree_rate = 0.05
volatility = 0.25
strike_price = 0.5

# Time limits
T_min = 0.
T_max  = 1.

# Space limits
S_min = 1e-10 
S_max = 1.

# Network parameters
nr_layers = 3
nr_nodes_per_layer = 50
initial_learning_rate = 0.001
learning_rate_decay_steps = 10000
learning_rate_decay_rate = 0.9

# Training parameters
steps_per_sample = 10
nr_epochs = 1000

# Number of samples
N_interior = 10000
N_initial = 10000

In [3]:
# DGM neural network model
class DNN(tf.keras.Model):#creating a class called DNN
    def __init__(self, nr_layers, nr_nodes_each_layer, state_dimension=1):#init is similiar to a constructor in c++
        #self allows you to call instances of that class similar to a this pointer.
        tf.keras.Model.__init__(self)#calls the parent constructor
        self.nr_layers = nr_layers #assinging member variables nr_layers as input varaible 

        self.initial_layer = DenseLayer(state_dimension + 1, nr_nodes_each_layer, activation=tf.nn.tanh)
        #setting parameters for initial layers start from 2(state dimension +time(1)) nodes all the way to 50 nodes
        self.hidden_layers = []
        for _ in range(nr_layers): #iterating over the 3 layers
            self.hidden_layers.append(LayerFromPaper(state_dimension + 1, nr_nodes_each_layer, activation=tf.nn.tanh))#appending the hidden layers create 3 of them
        self.final_layer = DenseLayer(nr_nodes_each_layer, 1, activation=None)#create the last layer


    def call(self, t, x):# creating of a member function
        X = tf.concat([t,x], 1) # concats the time and stock price in columns

        S = self.initial_layer.call(X)#call is a member function of dense layer
        for i in range(self.nr_layers):
            S = self.hidden_layers[i].call({'S': S, 'X': X})#creating the hidden layers, #X=time and asset price we concat this in  X = tf.concat([t,x], 1)
        result = self.final_layer.call(S)#creating the final layer

        return result
    


# Neural network layers

class DenseLayer(tf.keras.layers.Layer):# creating the class Dense layers
    def __init__(self, nr_inputs, nr_outputs, activation): #creating the constructor for that class
        tf.keras.layers.Layer.__init__(self)#initialzing the object of that class
        
        self.initializer = tf.keras.initializers.glorot_normal
        #self.initializer=tf.contrib.layers.xavier_initializer()) #TF 1

        self.nr_inputs = nr_inputs# initilaizing nr_inputs as a member variable
        self.nr_outputs = nr_outputs # initilizing nr_outputs as a member varaible
        
        self.W = self.add_variable("W", shape=[self.nr_inputs, self.nr_outputs],
                                   initializer=self.initializer())# initializing W as a member variable creating a matrix type object in order to train it for the neural network
        #W is one of the weights
        self.b = self.add_variable("b", shape=[1, self.nr_outputs])  #bias or constant added only at the end of training as therefore has no initilization

        self.activation = activation #saving it as a member variable
    
    
    def call(self, inputs):#member function of Dense Layer
        S = tf.add(tf.matmul(inputs, self.W), self.b) #From paper
        if not self.activation == None:
            S = self.activation(S) #activation function with the sigma (not volatility)

        return S



class LayerFromPaper(tf.keras.layers.Layer):
    def __init__(self, nr_inputs, nr_outputs, activation):
        tf.keras.layers.Layer.__init__(self)

        self.initializer = tf.keras.initializers.glorot_normal
        
        self.nr_outputs = nr_outputs
        self.nr_inputs = nr_inputs

        self.Uz = self.add_variable("Uz", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Ug = self.add_variable("Ug", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Ur = self.add_variable("Ur", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Uh = self.add_variable("Uh", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wz = self.add_variable("Wz", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wg = self.add_variable("Wg", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wr = self.add_variable("Wr", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wh = self.add_variable("Wh", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.bz = self.add_variable("bz", shape=[1, self.nr_outputs])
        self.bg = self.add_variable("bg", shape=[1, self.nr_outputs])
        self.br = self.add_variable("br", shape=[1, self.nr_outputs])
        self.bh = self.add_variable("bh", shape=[1, self.nr_outputs])

        self.activation = activation

    
    def call(self, inputs):
        S = inputs['S']
        X = inputs['X']

        Z = self.activation(tf.add(tf.add(tf.matmul(X, self.Uz), tf.matmul(S, self.Wz)), self.bz))
        G = self.activation(tf.add(tf.add(tf.matmul(X, self.Ug), tf.matmul(S, self.Wg)), self.bg))
        R = self.activation(tf.add(tf.add(tf.matmul(X, self.Ur), tf.matmul(S, self.Wr)), self.br))
        H = self.activation(tf.add(tf.add(tf.matmul(X, self.Uh), tf.matmul(tf.multiply(S, R), self.Wh)), self.bh))
        Snew = tf.add(tf.multiply(tf.subtract(tf.ones_like(G), G), H), tf.multiply(Z, S))

        return Snew

In [4]:
# Loss function
def get_residual(model, t_interior, x_interior, t_initial, x_initial):#model =BS Model,t_interior=time from 0 to 1,
    #x_interior=stock price from min to max, t_initial =t_min, x_initial=S_min
    # Loss term #1: PDE
    V = model(t_interior, x_interior)# valuation of the neural network
    V_t = tf.gradients(V, t_interior)[0] # DV/DT
    V_x = tf.gradients(V, x_interior)[0] # DV/DS
    V_xx = tf.gradients(V_x, x_interior)[0] #D^2V/DS^2
    #black scholes formula
    f = -V_t + riskfree_rate * V -riskfree_rate*x_interior*V_x - 0.5*volatility**2 * x_interior**2 * V_xx

    L1 = tf.reduce_mean(tf.square(f)) #mean of the squared residuals, residuals of the PDE J(F) part 1
    #Payoff function
    payoff= tf.math.maximum(0.,tf.subtract(strike_price,x_interior))
    #L2 norm
    L2=tf.reduce_mean(tf.square(tf.math.maximum(0.,payoff-V)))#J(f) part 2
    #max deviation of L2
    Max_dev=tf.math.reduce_max(tf.math.maximum(0.,payoff-V))
    #min deviation of L2
    Min_dev=tf.math.reduce_min(tf.math.maximum(0.,payoff-V))
    # Loss term #3: initial/terminal condition
    L3 = tf.reduce_mean(tf.square(model(t_initial,x_initial) - tf.math.maximum(0., strike_price - x_initial))) # J(F) part 3

    return (Max_dev,L1,L2,L3)


In [5]:
#  Sampling
def get_monte_carlo_points(N_interior, N_initial):
    # Sampler #1: PDE domain
    t_interior = np.random.uniform(low=T_min - 0.5*(T_max - T_min),
                           high=T_max,
                           size=[N_interior,1])
    s_interior = np.random.uniform(low=S_min - (S_max - S_min)*0.5,
                           high=S_max + (S_max - S_min)*0.5,
                           size=[N_interior,1])
    #you take all the t and the state space
    
    # Sampler #2: initial/terminal condition
    t_initial = T_max * np.ones((N_initial,1)) #Terminal condition
    s_initial = np.random.uniform(low=S_min - (S_max - S_min)*0.5,
                           high=S_max + (S_max - S_min)*0.5,
                           size=[N_initial,1])
    
    return (t_interior, s_interior, t_initial, s_initial)

In [6]:
nn_plot_list=[]
def compute_errors():
    max_error=0
    times_to_evaluate_error=np.linspace(T_min,T_max,10)
    for t in times_to_evaluate_error:
        tt = t*np.ones_like(xplot.reshape(-1,1))
    
        nn_plot, = sess.run([vplot_t],
                        feed_dict={tplot_t:tt, xplot_t:xplot.reshape(-1,1)})
        nn_plot_list.append(nn_plot,)
    
    return nn_plot_list

In [7]:
# Neural Network definition

model = DNN(nr_layers, nr_nodes_per_layer) #first time this model is constructed

t_interior_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="time_interior") #allows you to fill it with numbers
x_interior_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, dimension_state), name="stock_prices_interior")
t_initial_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="time_initial")#allows you to fill it with numbers
x_initial_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, dimension_state), name="stock_prices_initial")


Max_dev,residual_interior,residual_exterior,residual_initial= get_residual(model, t_interior_tf, x_interior_tf, t_initial_tf, x_initial_tf)
residual = residual_interior + residual_initial+(residual_exterior*1) #this residual is the combination of the L1 and L2 norm
# Optimizer parameters
nr_steps = tf.Variable(0, trainable=False) #very weird itertation counter, counts the no of optimization steps we took
learning_rate = tf.compat.v1.train.exponential_decay(initial_learning_rate, nr_steps,
                                           learning_rate_decay_steps, 
                                           learning_rate_decay_rate, staircase=True)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(residual) #defining the optimizer 
# gradient descent with a momemtum term. mizimised the residuals

# Plot tensors
tplot_t = tf.compat.v1.placeholder(tf.float32, [None,1], name="tplot_t") # We name to recover it later
xplot_t = tf.compat.v1.placeholder(tf.float32, [None,1], name="xplot_t")
vplot_t = tf.identity(model(tplot_t, xplot_t), name="vplot_t") # Trick for naming the trained model

init_op = tf.compat.v1.global_variables_initializer()




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
# before opening a tensorflow session, close the old one if possible

try:
    sess.close()
except NameError:
    pass 
sess =  tf.compat.v1.Session()
sess.run(init_op)

for epoch in range(nr_epochs):
    t_interior_mc, x_interior_mc, t_initial_mc, x_initial_mc = get_monte_carlo_points(N_interior, N_initial)

    for _ in range(steps_per_sample):
          Max_deviation,residual_value, residual_interior_value,residual_exterior_value,residual_initial_value, _ = \
              sess.run([Max_dev,residual, residual_interior, residual_exterior,residual_initial, optimizer],
                      feed_dict = {t_interior_tf:t_interior_mc, x_interior_tf:x_interior_mc,
                                    t_initial_tf:t_initial_mc, x_initial_tf:x_initial_mc})
        
    if (not np.mod(epoch, 100)) or epoch+1==nr_epochs:
        print("Stage: {:04d}, Loss: {:e}, Maximum_Deviation: {:e}, L1: {:e}, L2: {:e}, L3: {:e}".format(
            epoch,residual_value,Max_deviation, residual_interior_value,residual_exterior_value,residual_initial_value) )

Stage: 0000, Loss: 3.351902e-01, Maximum_Deviation: 8.994368e-01, L1: 1.545323e-01, L2: 3.558923e-02, L3: 1.450686e-01
Stage: 0100, Loss: 4.011974e-04, Maximum_Deviation: 4.318166e-02, L1: 6.813802e-05, L2: 8.615850e-05, L3: 2.469008e-04
Stage: 0200, Loss: 1.980041e-04, Maximum_Deviation: 2.994069e-02, L1: 4.174039e-05, L2: 4.840116e-05, L3: 1.078625e-04
Stage: 0300, Loss: 1.479570e-04, Maximum_Deviation: 2.466556e-02, L1: 3.075591e-05, L2: 4.076782e-05, L3: 7.643325e-05
Stage: 0400, Loss: 1.261060e-04, Maximum_Deviation: 2.434850e-02, L1: 2.469985e-05, L2: 4.056476e-05, L3: 6.084137e-05
Stage: 0500, Loss: 1.072655e-04, Maximum_Deviation: 2.281111e-02, L1: 2.708731e-05, L2: 3.835243e-05, L3: 4.182577e-05
Stage: 0600, Loss: 9.721018e-05, Maximum_Deviation: 2.256328e-02, L1: 2.603231e-05, L2: 3.778138e-05, L3: 3.339649e-05
Stage: 0700, Loss: 8.543250e-05, Maximum_Deviation: 2.202976e-02, L1: 2.544727e-05, L2: 3.595691e-05, L3: 2.402832e-05
Stage: 0800, Loss: 7.901820e-05, Maximum_Deviati

In [9]:
# Plot results
N = 41      # Points on plot grid

times_to_plot = [0*T_max, 0.33*T_max, 0.66*T_max, T_max]
tplot = np.linspace(T_min, T_max, N) # for surface plot
xplot = np.linspace(S_min, S_max, N)


In [10]:
nn_plot_list=compute_errors()
nn_plot_container = np.transpose(np.reshape(nn_plot_list,(-1,41)))
nn_plots = pd.DataFrame(nn_plot_container)
nn_plots=nn_plots.rename(columns={0: 'T=0',1 : 'T=0.1111',2 : 'T=0.2222',3 : 'T=0.3333',4 : 'T=0.4444',5 : 'T=0.5555',6 : 'T=0.6666',7 : 'T=0.7777',8 : 'T=0.8888',9 : 'T=1'})
print(nn_plots)

         T=0  T=0.1111  T=0.2222  T=0.3333  T=0.4444  T=0.5555  T=0.6666  \
0   0.489975  0.491675  0.493298  0.494863  0.496391  0.497907  0.499440   
1   0.464882  0.466580  0.468199  0.469760  0.471283  0.472798  0.474334   
2   0.439896  0.441617  0.443256  0.444834  0.446374  0.447904  0.449459   
3   0.415001  0.416769  0.418452  0.420069  0.421646  0.423212  0.424798   
4   0.390162  0.391997  0.393745  0.395424  0.397058  0.398675  0.400308   
5   0.365328  0.367244  0.369073  0.370831  0.372540  0.374225  0.375918   
6   0.340437  0.342441  0.344359  0.346208  0.348004  0.349771  0.351536   
7   0.315435  0.317517  0.319522  0.321462  0.323352  0.325209  0.327053   
8   0.290280  0.292417  0.294489  0.296509  0.298486  0.300431  0.302356   
9   0.264969  0.267115  0.269216  0.271284  0.273324  0.275343  0.277342   
10  0.239548  0.241638  0.243707  0.245766  0.247823  0.249879  0.251927   
11  0.214143  0.216088  0.218037  0.220006  0.222005  0.224034  0.226081   
12  0.188969